<font color=gray>Oracle Data Science service sample notebook.

Copyright (c) 2023 Oracle, Inc.  All rights reserved.
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

***
# <font color=red>Deploy LangChain Application as OCI Data Science Model Deployment</font>
<p style="margin-left:10%; margin-right:10%;">by the <font color=teal> Oracle Cloud Infrastructure Data Science Service Team </font></p>

***

## Overview

The notebook demonstrates how to deploy LangChain application as OCI Data Science Model Deployment using Oracle Accelerated Data Science (ADS) SDK.

The `ChainDeployment` class in ADS allows you to rapidly get a LangChain application into production. The `.prepare()` method serializes the LangChain application as `chain.yaml` file and generates `score.py` file which can further be uploaded to OCI model catalog. The uploaded model can be subsequently deployed into production.

Compatible conda pack: [pytorch21_p39_gpu_v1](https://docs.oracle.com/en-us/iaas/data-science/using/conda-gml-fam.htm) for CPU on Python 3.9 (version 1.0)

### Prequisites

This notebook requires authorization to work with the OCI Data Science Service. Details can be found [here](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/cli/authentication.html#). For the purposes of this notebook what is important to to know is that resource principals will be used absent api_key authentication.

---

## Contents

* <a href='#intro'>Introduction</a>
* <a href='#create'>Create a LangChain Application</a>
* <a href='#deploy'>Deploy LangChain Application as OCI Model Deployment</a>
    * <a href='#deploy_chaindeployment'>Create a ChainDeployment</a>
    * <a href='#deploy_prepare'>Prepare</a>
    * <a href='#deploy_verify'>Verify</a>
    * <a href='#deploy_save'>Save</a>
    * <a href='#deploy_deploy'>Deploy</a>
    * <a href='#deploy_predict'>Predict</a>
* <a href='#clean_up'>Clean Up</a>
* <a href='#ref'>References</a>    

---

In [ ]:
import ads
import os
import tempfile

from ads.llm.deploy import ChainDeployment
from langchain.llms import Cohere
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from shutil import rmtree

<a id='intro'></a>
# Introduction

In this notebook, you will create a custom LangChain application that links prompt and Cohere model and deploy it on OCI model deployment. It is designed to demonstrate how to use the `ChainDeployment` class in Oracle ADS SDK.

The `.prepare()` method will serialize the LangChain application as `chain.yaml` file. It will also generate a `score.py` file that will load the LangChain yaml and call the `predict()` method. The `.save()` and `.deploy()` methods will upload the artifacts to OCI model catalog and deploy the uploaded model, respectively.

### Authenticate

Authentication to the OCI Data Science service is required. Here we default to resource principals.

In [ ]:
ads.set_auth(auth="resource_principal")

<a id='create'></a>
# Create a LangChain Application

The next cell creates a LangChain application that links prompt and Cohere model. The LangChain application will utilize Cohere model to generate a joke based on the subject that user provides. Remember to replace the `<cohere_api_key>` with the actual api key as Cohere model needs it. You can acquire this key by registering on [Cohere](https://dashboard.cohere.com/welcome/register).

In [ ]:
os.environ["COHERE_API_KEY"] = "<cohere_api_key>"

cohere = Cohere() 
prompt = PromptTemplate.from_template("Tell me a joke about {subject}")
llm_chain = LLMChain(prompt=prompt, llm=cohere, verbose=True)

Now you have a LangChain object `llm_chain`. Try running it with the prompt `{"subject": "animals"}` and it should give you the corresponding answer.

In [ ]:
llm_chain.run({"subject": "animals"})

<a id='deploy'></a>
# Deploy LangChain Application as OCI Model Deployment

<a id='deploy_chaindeployment'></a>
## Create a ChainDeployment

The next cell creates a model artifact directory. This directory is used to store the artifacts that are needed to deploy the model. It also creates the `ChainDeployment` object. The `ChainDeployment` requires the LangChain object `llm_chain` as `chain` parameter.

In [ ]:
artifact_dir = tempfile.mkdtemp()

chain_deployment = ChainDeployment(
    chain=llm_chain,
    artifact_dir=artifact_dir
)

<a id='deploy_prepare'></a>
## Prepare

The prepare step is performed by the `.prepare()` method of the `ChainDeployment` class. It creates a number of customized files that are used to run the model once it is deployed. These include:

* `chain.yaml`: A YAML file that is serialized from the LangChain application and can be deserialized in `load_model` in `score.py`.
* `runtime.yaml`: This file contains information that is needed to set up the runtime environment on the deployment server.
* `score.py`: This script contains the `load_model()` and `predict()` functions. The `load_model()` function understands the format the model file was saved in, and loads it into memory. The `.predict()` method is used to make inferences in a deployed model.

To create the model artifacts, you use the `.prepare()` method

* `inference_conda_env` variable defines the slug of the conda environment that is used to train the model

Note that you can only pass in slug for service conda environment. For custom conda environment, you have to pass in the full path along with the `inference_python_version`. 

Here, replace `<custom_conda_environment_uri>` with your conda environment uri that has the latest ADS 2.9.1 and replace `<python_version>` with your conda environment python version. For how to customize and publish conda environment, take reference to [Publishing a Conda Environment to an Object Storage Bucket](https://docs.oracle.com/en-us/iaas/data-science/using/conda_publishs_object.htm).

In [ ]:
chain_deployment.prepare(
    inference_conda_env="<custom_conda_environment_uri>",
    inference_python_version="<python_version>",
)

<a id='deploy_verify'></a>
## Verify

The `.verify()` method takes a set of test parameters and performs the prediction by calling the `predict` function in `score.py`. It also runs the `load_model` function.

In [ ]:
chain_deployment.verify({"subject": "animals"})

<a id='deploy_save'></a>
## Save

Call `.save()` to pack and upload the artifacts under `artifact_dir` to OCI data science model catalog. Once the artifacts are successfully uploaded, you should be able to see the id of the model.

In [ ]:
chain_deployment.save(display_name="LangChain Model")

<a id='deploy_deploy'></a>
## Deploy

Deploy the LangChain model from previous step by calling `.deploy()`. For more information regarding the allowed parameters, see [here](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/model_serialization/genericmodel.html#deploy). Remember to replace the `<cohere_api_key>` with the actual cohere api key in the `environment_variables`. It usually takes a couple of minutes to deploy the model and you should see the model deployment in the output once the process completes.

In [ ]:
chain_deployment.deploy(
    display_name="LangChain Model Deployment",
    environment_variables={"COHERE_API_KEY":"<cohere_api_key>"}, 
)

<a id='deploy_predict'></a>
## Predict

After the deployment is active, you can call the `predict()` on the `ChainDeployment` object to send request to the deployed endpoint. 

In [ ]:
chain_deployment.predict(data={"subject": "animals"})

<a id='clean_up'></a>
# Clean Up

This notebook created a model deployment and a model. This section deletes those resources. 

The model deployment must be deleted before the model can be deleted. You can use the `.delete_deployment()` method on the `ChainDeployment` object to do this.

In [ ]:
delete = chain_deployment.delete_deployment(wait_for_completion=True)

Use the `.delete()` method to delete the model:

In [ ]:
chain_deployment.delete()

The next cell removes the model artifacts that were stored on your local drive:

In [ ]:
rmtree(artifact_dir)

<a id='ref'></a>
# References
- [ADS Library Documentation](https://accelerated-data-science.readthedocs.io/en/latest/index.html)
- [Data Science YouTube Videos](https://www.youtube.com/playlist?list=PLKCk3OyNwIzv6CWMhvqSB_8MLJIZdO80L)
- [OCI Data Science Documentation](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm)
- [Oracle Data & AI Blog](https://blogs.oracle.com/datascience/)
- [Understanding Conda Environments](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/use-notebook-sessions.htm#conda_understand_environments)
- [Use Resource Manager to Configure Your Tenancy for Data Science](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/orm-configure-tenancy.htm)
- [runtime.yaml](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_runtime_yaml.htm#model_runtime_yaml)
- [score.py](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/model_score_py.htm#model_score_py)
- [Model artifact](https://docs.content.oci.oracleiaas.com/en-us/iaas/data-science/using/models_saving_catalog.htm#create-models)